## Credit Risk Project

### Cleaning the Data

Below we read in the csv file. We drop the description column, whichi we not easily make use of, as well as any column missing more than 50% of its values.

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.linear_model import LogisticRegression

loans_2007 = pd.read_csv('LoanStats3a.csv', skiprows=1,sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
half_count = len(loans_2007) / 2
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)
loans_2007 = loans_2007.drop('desc',axis=1)


In [2]:
loans_2007.head()


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,hardship_flag,disbursement_method,debt_settlement_flag
0,5000,5000,4975,36 months,10.65%,162.87,B,B2,NaN,10+ years,...,1,Individual,0,0,0,0,0,N,Cash,N
1,2500,2500,2500,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,...,1,Individual,0,0,0,0,0,N,Cash,N
2,2400,2400,2400,36 months,15.96%,84.33,C,C5,NaN,10+ years,...,1,Individual,0,0,0,0,0,N,Cash,N
3,10000,10000,10000,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,...,1,Individual,0,0,0,0,0,N,Cash,N
4,3000,3000,3000,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,...,1,Individual,0,0,0,0,0,N,Cash,N


We continue to drop columns. We will remove columns that leak the loan status, are irrelevant (such as loan IDs), or are redundant. In the case of redundant columns, we will retain those that are continuous over categorical, as they are more suitable for machine learning. 

In [3]:
loans_2007 = loans_2007.drop(['funded_amnt', 'funded_amnt_inv','grade','sub_grade', 'emp_title', 'issue_d','debt_settlement_flag'],axis=1)

In [4]:
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)

In [5]:
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print(loans_2007.shape[1])

34


We will be attempting to predict if a loan is paid off or not. This information is contained in the loan_status column. We can check the values that exist in the column. Charged Off means the loan is not expected to be paid back. We first remove any row where the loan is not either full paid off or charged off. We then replace the column with a binary value representing if the loan was paid or not.

In [6]:
print(loans_2007["loan_status"].value_counts())

Fully Paid                                             34116
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64


In [7]:
mapdict = {"Fully Paid":1, "Charged Off":0}

loans_2007 = loans_2007[loans_2007["loan_status"].isin(['Fully Paid', 'Charged Off'])]
                        
loans_2007 = loans_2007.replace(mapdict)

We next remove any columns that only have one unique value. These columns have no variance and cannot contribute to our machine learning algorithm.

In [8]:
drop_columns=[]

listcols=loans_2007.columns.tolist()

for col in listcols:
    
    if len(loans_2007[col].dropna().unique()) ==1:
        drop_columns.append(col)
        
        
loans=loans_2007.drop(drop_columns,axis=1)

print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens', 'hardship_flag', 'disbursement_method']


Checking to see how many nulls are in each column.

In [9]:
null_counts = loans.isnull().sum()

print(null_counts)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


We can devise a rule, for example, to drop any column with >1% missing values. In this scenario, we would drop the column pub_rec_bankruptcies.

In [10]:
loans = loans.drop('pub_rec_bankruptcies', axis=1)

loans = loans.dropna()

loans.dtypes.value_counts()

object    21
int64      1
dtype: int64

### Preparing the Features

The object columns that contain text will need to be converted to numerical values to be useful to us. We can investigate what values each categorical column contains.

In [11]:
object_columns_df = loans.select_dtypes(include=['object'])

print(object_columns_df.head(1))

  loan_amnt        term int_rate installment emp_length home_ownership  \
0      5000   36 months   10.65%      162.87  10+ years           RENT   

  annual_inc verification_status      purpose     title        ...          \
0      24000            Verified  credit_card  Computer        ...           

     dti delinq_2yrs earliest_cr_line inq_last_6mths open_acc pub_rec  \
0  27.65           0         Jan-1985              1        3       0   

  revol_bal revol_util total_acc last_credit_pull_d  
0     13648      83.7%         9           Feb-2018  

[1 rows x 21 columns]


Below we look at some categorical columns. We will represent all of them, except for addr_state which has too many values, with dummy columns.

In [13]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']


for col in cols:

    print(loans[col].value_counts())

RENT        18881
MORTGAGE    17688
OWN          3056
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16890
Verified           12833
Source Verified    10001
Name: verification_status, dtype: int64
10+ years    8897
< 1 year     4576
2 years      4389
3 years      4094
4 years      3435
5 years      3279
1 year       3240
6 years      2227
7 years      1771
8 years      1483
9 years      1259
n/a          1074
Name: emp_length, dtype: int64
 36 months    29041
 60 months    10683
Name: term, dtype: int64
CA    7095
NY    3815
FL    2869
TX    2729
NJ    1850
IL    1524
PA    1515
VA    1407
GA    1399
MA    1343
OH    1221
MD    1053
AZ     878
WA     841
CO     791
NC     788
CT     754
MI     722
MO     685
MN     613
NV     497
SC     472
WI     459
AL     451
OR     450
LA     436
KY     327
OK     299
KS     271
UT     259
AR     245
DC     212
RI     199
NM     190
WV     177
HI     173
NH     172
DE     113
MT      85
WY      83
AK      

The purpose and title columns seem to be redundant. We will choose to keep the purpose columnn since it contains less discrete values. 

In [14]:
print(loans["purpose"].value_counts())

print(loans["title"].value_counts())

debt_consolidation    18661
credit_card            5134
other                  3985
home_improvement       2980
major_purchase         2182
small_business         1827
car                    1549
wedding                 947
medical                 693
moving                  581
house                   382
vacation                380
educational             320
renewable_energy        103
Name: purpose, dtype: int64
Debt Consolidation                        2189
Debt Consolidation Loan                   1732
Personal Loan                              661
Consolidation                              516
debt consolidation                         508
Credit Card Consolidation                  357
Home Improvement                           357
Debt consolidation                         334
Small Business Loan                        329
Credit Card Loan                           319
Personal                                   309
Consolidation Loan                         256
Home Improvement

Meanwhile, we will simplify the emp_length column by capping the values at 10+ years, and counting all missing values as 0. We will also drop the last_credit_pull_d and earliest_cr_line columns, which will require too much processing to be useful.

In [15]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}


loans = loans.drop(["last_credit_pull_d", "addr_state", "title", "earliest_cr_line"],axis=1)

loans['int_rate']=loans['int_rate'].str.rstrip('%').astype('float')
loans['revol_util']=loans['revol_util'].str.rstrip('%').astype('float')

loans=loans.replace(mapping_dict)


We create dummy columns, concatenate them to our original dataframe, and drop the corresponding originals.

In [16]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

In [17]:
pd.set_option('display.max_columns', None)

In [29]:
loans.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000,10.65,162.87,10,24000,1,27.65,0,1,3,0,13648,83.7,9,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2500,15.27,59.83,0,30000,0,1,0,5,3,0,1687,9.4,4,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2400,15.96,84.33,10,12252,1,8.72,0,2,2,0,2956,98.5,10,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,10000,13.49,339.31,10,49200,1,20,0,1,10,0,5598,21.0,37,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,3000,12.69,67.79,1,80000,1,17.94,0,0,15,0,27783,53.9,38,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


### The Algorithm

Separating loans into our features and our target.

In [26]:
features = loans.drop("loan_status", axis=1)

target = loans["loan_status"]


Our first attempt will be a naive application, not taking into the unbalanced nature of the data. We can see our model predicts almost entirely 1s. This results in a lot of false positives, which in our case is agreeing to a loan with a borrower who will not pay back on time. As a conservative investor, this is the situation we wanted to avoid and yet our model does almost nothing to help us. 

In [40]:
# %load algorithm_unbalanced.py
#This is the logistic regression algorithm. The data is unbalanced, and have much more loans that are paid off than are defaulted on. This results in a large false positive rate. 

# Importing necessary modules
import pandas as pd
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
kf = KFold(features.shape[0])


predictions = cross_val_predict(lr, features,target,cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# True positive and false positive rates
tpr = float(tp) / float(tp + fn)
fpr = float(fp) / float(fp + tn)

print(tpr)
print(fpr)


0.998167379352
0.997245179063


We now try including different punishments for misclassification. We now see that our false positive rate is now 2%. Meaning, we avoid 98% of borrowers who default. This is much better for the conservative investor. However, we can see that it has come at the cost of missing responsible borrowers, who we now miss around 93% of. 

In [38]:
# %load algorithm_penalty.py
#This is the logistic regression algorithm. Here we impose a penalty of 10 for misclassifying a 0, and a penalty of 1 for misclassifying a 1. This is compensate for the fact that we are looking at an unbalanced data set. 

# Importing necessary packages
import pandas as pd
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.linear_model import LogisticRegression

penalty = {0:10, 1:1} #The penalty dictionary

lr = LogisticRegression(class_weight=penalty)

kf = KFold(features.shape[0])

predictions = cross_val_predict(lr, features,target, cv=kf)

predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = float(tp) / float(tp + fn)  # true positive rate
fpr = float(fp) / float(fp + tn)  # false positive rate 

print(tpr)
print(fpr)


0.0867440439829
0.0247933884298
